## Пример на Keras более сложной сверточной нейронной сети

> Indented block



Давайте теперь попробуем сделать несколько усложненный вариант нейронной сети разобранной ранее. В ней будет на несколько слоев больше и в ней будет использоваться data augumentation, процедура позволяющая за счет искажений изображений увеличить количество тренировочных данных, а как мы знаем чем больше тренировочных данных тем лучше будет работать нейросеть. Для обучения нейросети будем использовать датасет cifar-10. В нем 10 категорий объектов, например - лошадь, лягушка, корабль. Данный датасет уже более сложен для нейронных сетей чем mnist, однако он намного проще датасетов наподобие imagenet где используются сотни классов и архитектуры нейронных сетей для подобных датасетов также понадобяться более сложные.

In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

In [2]:
# Зададим параметры нейронной сети
batch_size = 3
num_classes = 10
epochs = 3

# Различные варианты изменения изображения
data_augmentation = True

num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_mnist_trained_model.h5'

In [3]:
# Загружаем датасет MNISt
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'training examples')
print(x_test.shape[0], 'testing examples')

x_train shape: (60000, 28, 28)
60000 training examples
10000 testing examples


In [4]:
# Reshape the input data to have a rank-4 shape
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)

# Преобразование матрицы чисел 0-9 в бинарную матрицу чисел 0-1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [5]:
# Define the model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation="relu", padding="same", input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation="relu", padding="same"),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation="relu", padding="same"),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation="relu"),
    Dense(10, activation="softmax")
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 128)         73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 3, 3, 128)        0

In [6]:
# Compile the model
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(),
              metrics=['accuracy'])

In [7]:
# обучение модели без аугментации
if not data_augmentation:
    print('Not using data augmentation')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation')
#  Обучение модели с аугментацией, дата-генератором изображений
    # Создаем генератор с определенными настройками
    datagen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False,
        vertical_flip=False)

    # обучаем генератор на обучающей выборке, тестовую не подаем
    # datagen.fit(x_train)

    # обучаем модель с учетом генератора
    model.fit(datagen.flow(x_train, y_train,
                           batch_size=batch_size),
              epochs=epochs,
              validation_data=(x_test, y_test))
    
model.fit_generator(datagen.fit(x_train, y_train,
                                batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)


Using real-time data augmentation
Epoch 1/3
20000/20000 [==============================] - 141s 7ms/step - loss: 0.8357 - accuracy: 0.7915 - val_loss: 0.2259 - val_accuracy: 0.9252
Epoch 2/3
20000/20000 [==============================] - 144s 7ms/step - loss: 0.3257 - accuracy: 0.8956 - val_loss: 0.2362 - val_accuracy: 0.9247
Epoch 3/3
20000/20000 [==============================] - 144s 7ms/step - loss: 0.3087 - accuracy: 0.9038 - val_loss: 0.1693 - val_accuracy: 0.9454


In [8]:
# Save the trained model сохранение модели в память
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at', model_path)

# Evaluate the model on the test data
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Saved trained model at c:\Users\Lenovo\MyPython\Neural_Network\les_4\saved_models\keras_mnist_trained_model.h5
313/313 [==============================] - 4s 12ms/step - loss: 0.1693 - accuracy: 0.9454
Test loss: 0.1693032681941986
Test accuracy: 0.9453999996185303
